# Python

# stdlib

- standard library 
- great data types and operator overloading
- great program constructs: much more of a "coding" or "programming" language relative to R

In [ ]:
#  e.g., loops, dictionaries
# e.g., strings are super easy to use like "arrays" or lists

x=[]
y=""
z=dict()
n=10
for i in range(n):
    x += [i]
    y += str(i)
    z[i]=i
    
print(len(x), x, type(x))
print(len(y), y, type(y))
print(len(z), z, type(z))

In [ ]:
# e.g., functions, recurssion, conditionals

def fibonacci(sequence=[0,1], steps=6):
    if steps==0:
        return sequence
    else:
        return fibonacci(sequence+[sum(sequence[-2:])], steps=steps-1)
    
print(fibonacci())

# numpy
- 3-4x slower than C++
- ndarrays, broadcasting

In [ ]:
import numpy as np

In [ ]:
n=1000000

In [ ]:
%%timeit
_ = np.arange(n)*np.arange(n)

In [ ]:
%%timeit
_ = [i*j for i,j in zip(range(n),range(n))]

In [ ]:
print(np.zeros(10).reshape(5,2,1,1))


In [ ]:
print(np.ones([1,10]))
print(np.ones([10,1]))

np.ones([1,10])*np.ones([10,1])

# scipy

$$\huge \int_{-1}^{1} \frac{1}{\sqrt{2\pi}}e^{-\frac{x^2}{2}} dx$$


In [ ]:
import scipy
from scipy import stats

print(scipy.integrate.quad(lambda x: stats.norm.pdf(x), -1, 1))

scipy.special.erf(1/np.sqrt(2)), 2*(stats.norm.cdf(1)-.5)


In [ ]:
scipy.special.erf?

In [ ]:
n = 1000
print(stats.norm(loc=1, scale=2).rvs(n))

stats.norm.fit(stats.norm(loc=1, scale=2).rvs(n))

# pandas

- DataFrame functionality
    - everything is "methods" based, though
- "SQL"-like capabilities
- design focusses on timeseries (and correspondingly, indexes)

In [ ]:
import pandas as pd

dataset = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv'
penguins = pd.read_csv(dataset, dtype={key:'category' for key in ['species','island','sex']})
penguins.head()

In [ ]:
penguins.info()

In [ ]:
# like the R `summrary()` function
penguins.describe()

In [ ]:
import matplotlib.pyplot as plt
tabs = []
fig = plt.figure(figsize=[8,4])
ax = fig.subplots(2,2).flatten()
i = 0
for col in penguins:
    if penguins.dtypes[col]=='float64':
        penguins[[col]].plot(kind='kde', ax=ax[i])
        i+=1
    elif penguins.dtypes[col]=='category':
        tabs += [pd.DataFrame(penguins[col].value_counts(), columns=[col])]
    else:
        pass

plt.tight_layout()
plt.show()
_=[display(tab) for tab in tabs]


In [ ]:
strikes_path = 'http://lib.stat.cmu.edu/datasets/strikes'
strikes = pd.read_csv(strikes_path, skiprows=18, header=None,
                      delim_whitespace=True)
strikes.columns = ['Country', 'Year', 'Stike_Volume', 'Unemplyment',
                    'Inflation', 'Socialist_Party_Representation',
                    'Unionization']
strikes[['Year']] = strikes[['Year']].astype(type(0))

strikes.head()

In [ ]:
SnP_path = 'http://lib.stat.cmu.edu/datasets/spdc2693'
SnP = pd.read_csv(SnP_path, skiprows=5, header=None,
                  delim_whitespace=True)
SnP.columns = ['Date', 'Close']
SnP['Date'] = SnP.Date.apply(lambda x: pd.to_datetime('19'+str(x), format='%Y%m%d'))
SnP['Year'] = SnP.Date.apply(lambda x: x.year)

SnP.head()

In [ ]:
SnP_strikes = SnP.merge(strikes, on='Year', how='inner')
SnP_strikes.set_index('Date', inplace=True)
country_codes = {1.0: 'AusT',
                 2.0: 'AusA',
                 3.0: 'Bel',
                 4.0: 'Can',
                 5.0: 'Den',
                 6.0: 'Fin',
                 7.0: 'Fra',
                 8.0: 'Ger',
                 9.0: 'Ire',
                 10.0: 'Ita',
                 11.0: 'Jap',
                 12.0: 'Ned',
                 13.0: 'Nwz',
                 14.0: 'Nor',
                 15.0: 'Swe',
                 16.0: 'Swi',
                 17.0: 'UK',
                 18.0: 'Usa'}
SnP_strikes['Country'] = SnP_strikes.Country.map(country_codes)
strikes[['Country']] = pd.Categorical(strikes['Country'])

fig, ax = plt.subplots(2,1, figsize=(15,13))
#apply(lambda x: np.log(x) if x.name=='Stike_Volume' else x).\
SnP_strikes.groupby('Country').rolling(200).mean().reset_index(0).groupby('Country')['Unemplyment'].plot(ax=ax[0])
ax[0].set_title('Unemployment (%)')
ax[0].legend(loc='upper center')

SnP_strikes['Close'].plot(ax=ax[1])
ax[1].set_title('S&P 500 Closing Values')

# seaborn
- matplotlib 2.0

In [ ]:
SnP_strikes[SnP_strikes.Country.apply(lambda x: x in ['Usa','UK'])].\
    groupby(by=['Year','Country']).max()

In [ ]:
import seaborn as sns
SnP_strikes_US_UK = SnP_strikes[SnP_strikes.Country.apply(lambda x: x in ['Usa','UK'])].\
    groupby(by=['Year','Country']).max().reset_index(1).dropna()
#SnP_strikes_US_UK.Country.cat.remove_unused_categories(inplace=True)

p = sns.pairplot(SnP_strikes_US_UK.iloc[:,:-1], hue="Country")
p.map_lower(sns.kdeplot, levels=4, color=".2")
p.map_upper(sns.histplot)


# Scikit-Learn
- do Machine Learning with `sklearn`
- do stats with `pandas+statsmodels` (not shown)

In [ ]:
#https://scikit-learn.org/stable/auto_examples/gaussian_process/plot_gpc_iris.html#sphx-glr-auto-examples-gaussian-process-plot-gpc-iris-py
    
from sklearn import datasets    
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
y = np.array(iris.target, dtype=int)

h = .02  # step size in the mesh

kernel = 1.0 * RBF([1.0])
gpc_rbf_isotropic = GaussianProcessClassifier(kernel=kernel).fit(X, y)
kernel = 1.0 * RBF([1.0, 1.0])
gpc_rbf_anisotropic = GaussianProcessClassifier(kernel=kernel).fit(X, y)

# create a mesh to plot in
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

titles = ["Isotropic RBF", "Anisotropic RBF"]
plt.figure(figsize=(10, 5))
for i, clf in enumerate((gpc_rbf_isotropic, gpc_rbf_anisotropic)):
    # Plot the predicted probabilities. For that, we will assign a color to
    # each point in the mesh [x_min, m_max]x[y_min, y_max].
    plt.subplot(1, 2, i + 1)

    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape((xx.shape[0], xx.shape[1], 3))
    plt.imshow(Z, extent=(x_min, x_max, y_min, y_max), origin="lower")

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=np.array(["r", "g", "b"])[y],
                edgecolors=(0, 0, 0))
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())
    plt.title("%s, LML: %.3f" %
              (titles[i], clf.log_marginal_likelihood(clf.kernel_.theta)))

plt.tight_layout()

In [ ]:
#https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regularization.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-regularization-py
    
from sklearn import ensemble

X, y = datasets.make_hastie_10_2(n_samples=12000, random_state=1)
X = X.astype(np.float32)

# map labels from {-1, 1} to {0, 1}
labels, y = np.unique(y, return_inverse=True)

X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

original_params = {'n_estimators': 1000, 'max_leaf_nodes': 4, 'max_depth': None, 'random_state': 2,
                   'min_samples_split': 5}

plt.figure()

for label, color, setting in [('No shrinkage', 'orange',
                               {'learning_rate': 1.0, 'subsample': 1.0}),
                              ('learning_rate=0.1', 'turquoise',
                               {'learning_rate': 0.1, 'subsample': 1.0}),
                              ('subsample=0.5', 'blue',
                               {'learning_rate': 1.0, 'subsample': 0.5}),
                              ('learning_rate=0.1, subsample=0.5', 'gray',
                               {'learning_rate': 0.1, 'subsample': 0.5}),
                              ('learning_rate=0.1, max_features=2', 'magenta',
                               {'learning_rate': 0.1, 'max_features': 2})]:
    params = dict(original_params)
    params.update(setting)

    clf = ensemble.GradientBoostingClassifier(**params)
    clf.fit(X_train, y_train)

    # compute test set deviance
    test_deviance = np.zeros((params['n_estimators'],), dtype=np.float64)

    for i, y_pred in enumerate(clf.staged_decision_function(X_test)):
        # clf.loss_ assumes that y_test[i] in {0, 1}
        test_deviance[i] = clf.loss_(y_test, y_pred)

    plt.plot((np.arange(test_deviance.shape[0]) + 1)[::5], test_deviance[::5],
            '-', color=color, label=label)

plt.legend(loc='upper left')
plt.xlabel('Boosting Iterations')
plt.ylabel('Test Set Deviance')

# plotly
- primary language _is_ python

In [ ]:
# https://plotly.com/python/visualizing-mri-volume-slices/
# https://www.geeksforgeeks.org/animated-data-visualization-using-plotly-express/

import time
import numpy as np

from skimage import io

vol = io.imread("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/attention-mri.tif")
volume = vol.T
r, c = volume[0].shape

# Define frames
import plotly.graph_objects as go
nb_frames = 68

fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67 - k]),
    cmin=0, cmax=200
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=6.7 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67]),
    colorscale='Gray',
    cmin=0, cmax=200,
    colorbar=dict(thickness=20, ticklen=4)
    ))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()

In [ ]:
! yes y | conda install -c conda-forge ffmpeg


In [ ]:
! conda env list

In [ ]:
! conda activate base

In [ ]:
# https://matplotlib.org/3.1.0/gallery/animation/unchained.html

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib import animation, rc
rc('animation', html='html5')
%matplotlib notebook

# Fixing random state for reproducibility
np.random.seed(19680801)


# Create new Figure with black background
fig = plt.figure(figsize=(8, 8), facecolor='black')

# Add a subplot with no frame
ax = plt.subplot(111, frameon=False)

# Generate random data
data = np.random.uniform(0, 1, (64, 75))
X = np.linspace(-1, 1, data.shape[-1])
G = 1.5 * np.exp(-4 * X ** 2)

# Generate line plots
lines = []
for i in range(len(data)):
    # Small reduction of the X extents to get a cheap perspective effect
    xscale = 1 - i / 200.
    # Same for linewidth (thicker strokes on bottom)
    lw = 1.5 - i / 100.0
    line, = ax.plot(xscale * X, i + G * data[i], color="w", lw=lw)
    lines.append(line)

# Set y limit (or first line is cropped because of thickness)
ax.set_ylim(-1, 70)

# No ticks
ax.set_xticks([])
ax.set_yticks([])

# 2 part titles to get different font weights
ax.text(0.5, 1.0, "MATPLOTLIB ", transform=ax.transAxes,
        ha="right", va="bottom", color="w",
        family="sans-serif", fontweight="light", fontsize=16)
ax.text(0.5, 1.0, "UNCHAINED", transform=ax.transAxes,
        ha="left", va="bottom", color="w",
        family="sans-serif", fontweight="bold", fontsize=16)


def update(*args):
    # Shift all data to the right
    data[:, 1:] = data[:, :-1]

    # Fill-in new values
    data[:, 0] = np.random.uniform(0, 1, len(data))

    # Update data
    for i in range(len(data)):
        lines[i].set_ydata(i + G * data[i])

    # Return modified artists
    return lines

# Construct the animation, using the update function as the animation director.
anim = animation.FuncAnimation(fig, update, interval=50)
#HTML(anim.to_html5_video())
anim